In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import eigh
from scipy.sparse import kron, identity, csr_matrix
from scipy.sparse.linalg import eigsh  # sparse diagonalization
from tqdm import tqdm

In [ ]:
# Pauli matrices (dense)
I = np.eye(2)
sx = np.array([[0, 1], [1, 0]])
sz = np.array([[1, 0], [0, -1]])

def kron_N_exact(ops):
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)
    return result

def build_operator_exact(pauli, site, N):
    ops = [I] * N
    ops[site] = pauli
    return kron_N_exact(ops)

def tfim_hamiltonian_exact(N, J=1.0, h=1.0):
    dim = 2**N
    H = np.zeros((dim, dim), dtype=np.float64)

    for i in range(N-1):
        H -= J * np.dot(build_operator_exact(sz, i, N), build_operator_exact(sz, i+1, N))

    for i in range(N):
        H -= h * build_operator_exact(sx, i, N)

    return H

def solve_tfim_exact(N, J=1.0, h=1.0):
    """for N < 13 only"""
    H = tfim_hamiltonian_exact(N, J, h)
    eigvals, eigvecs = eigh(H)
    return eigvals, eigvecs

def run_tfim_scan_exact(N_range, h_values, J=1.0):
    results = []
    total_steps = len(N_range) * len(h_values)

    with tqdm(total=total_steps, desc="Running TFIM exact simulations") as pbar:
        for N in N_range:
            for h in h_values:
                eigvals, eigvecs = solve_tfim_exact(N=N, J=J, h=h)
                E0 = eigvals[0]
                results.append({"N": N, "h": h, "E0": E0})
                pbar.update(1)
    
    df = pd.DataFrame(results)
    
    return df

if __name__ == "__main__":
    h_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 10.0, 100.0] 
    
    N_range = range(2, 12)  # keep small for exact diag (2^N grows fast!)
    df1 = run_tfim_scan_exact(N_range=N_range, h_values=h_values)

In [ ]:
# Pauli matrices as sparse
sx = csr_matrix(np.array([[0, 1], [1, 0]]))
sz = csr_matrix(np.array([[1, 0], [0, -1]]))
I  = identity(2, format='csr')

def kron_N(ops):
    """Kronecker product of a list of operators"""
    result = ops[0]
    for op in ops[1:]:
        result = kron(result, op, format='csr')
    return result

def build_operator(pauli, site, N):
    """Build an N-qubit operator with `pauli` at `site`, Identity elsewhere"""
    ops = [I] * N
    ops[site] = pauli
    return kron_N(ops)
    
def tfim_hamiltonian(N, J=1.0, h=0.0):
    H = csr_matrix((2**N, 2**N), dtype=np.float64)

    # Interaction terms: -J * σ^z_i σ^z_{i+1}
    for i in range(N - 1):  # open boundary
        H -= J * build_operator(sz, i, N).dot(build_operator(sz, i + 1, N))

    # Field terms: -h * σ^x_i
    for i in range(N):
        H -= h * build_operator(sx, i, N)

    return H

def solve_tfim(N, J=1.0, h=1.0, k=1):
    H = tfim_hamiltonian(N, J, h)
    eigvals, eigvecs = eigsh(H, k, which='SA', tol=1e-12, maxiter=5000)  # Smallest algebraic
    return eigvals, eigvecs

def run_tfim_scan(N_range, h_values, J=1.0):
    results = []
    total_steps = len(N_range) * len(h_values)

    with tqdm(total=total_steps, desc="Running TFIM simulations") as pbar:
        for N in N_range:
            for h in h_values:
                eigvals, eigvecs = solve_tfim(N=N, J=J, h=h)
                E0 = eigvals[0]
                results.append({"N": N, "h": h, "E0": E0})
                pbar.update(1)

    df = pd.DataFrame(results)

    return df

if __name__ == "__main__":
    N_range = range(12, 20)
    h_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 10.0, 100.0] #[round(x * 0.1, 1) for x in range(11)]  + [10.0, 100.0]
    
    df2 = run_tfim_scan(N_range=N_range, h_values=h_values)

In [ ]:
df_results = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# Pivot the table
table = df_results.pivot(index="N", columns="h", values="E0")

# Reset index so 'N' becomes a normal column, not just the index
table = table.reset_index()

# Save to CSV with 'N' as the first named column
table.to_csv("tfim_ED_results.csv", index=False)

## Access ground state energy for a specific N in a list:

In [ ]:
E0_list = df_results[df_results["N"] == 4].sort_values("h")["E0"].tolist()
print(E0_list)

## Access all results for a specific N:

In [ ]:
pd.set_option('display.float_format', lambda x: f'{x:.16f}')
df_results[df_results["N"] == 4] 

## Access a single entry for specific N and h:

In [ ]:
df_results[(df_results["N"] == 4) & (df_results["h"] == 0.5)]

## Extract only E0 for specific N and h:

In [ ]:
E0_value = df_results[(df_results["N"] == 4) & (df_results["h"] == 1)]["E0"].values[0]
print(E0_value)